# **Read Images**


In [2]:
# reference: "https://github.com/kulikovv/DeepColoring"

from os import listdir
from os.path import join
import matplotlib.pyplot as plt
import cv2

In [3]:
path = "drive/My Drive/COMP9517/ALL/train"
imagesName = sorted([join(path,fn) for fn in listdir(path)])  # a list saving images name --> str list

In [4]:
images = [cv2.cvtColor(cv2.imread(fn), cv2.COLOR_BGR2RGB) for fn in imagesName if fn.endswith("rgb.png")] # a list saving RGB --> np float32 list

In [5]:
labels = [cv2.cvtColor(cv2.imread(fn), cv2.COLOR_BGR2GRAY) for fn in imagesName if fn.endswith("label.png")] # a list saving labels --> np float32 list

# **Pre-Process**

In [6]:
import cv2
import numpy as np
from skimage.transform import SimilarityTransform as st
import skimage.transform as transform
from skimage.filters import gaussian
from skimage.transform import warp

pre_process = []

In [7]:
#digitize：only labels need
def digitize(array, is_rgb):
    dst = array
    if not is_rgb:
        dst = np.digitize(array,bins=np.unique(array))-1
    return dst
pre_process.append(digitize)

In [8]:
#trandform_type rgb.astype(np.float32), label.astype(np.int32)
def trandform_type(array, is_rgb):
    if is_rgb:
        dst = array.astype("float32")
    else:
        dst = array.astype("int32")
    return dst
pre_process.append(trandform_type)

In [9]:
#clip the image to shape(192,192)
def get_part_random(array, is_rgb):
    cx = np.random.randint(0, array.shape[0] - 192)
    cy = np.random.randint(0, array.shape[1] - 192)
    return array[cx:cx+192, cy:cy+192]
pre_process.append(get_part_random)

In [10]:
def flip_way1(array, is_rgb):
    dst = array
    if np.random.random() < 0.5:
        dst = cv2.flip(array, 1)
    return dst
pre_process.append(flip_way1)

def flip_way2(array, is_rgb):
    dst = array
    if np.random.random() < 0.5:
        dst = cv2.flip(array, 0)
    return dst
pre_process.append(flip_way2)

def rotate90(array, is_rgb):
    dst = array
    if np.random.random() < 0.5:
        dst = np.rot90(array, 2, axes=(0, 1)) 
    return dst
pre_process.append(rotate90)

def gaussian_blur(array, is_rgb):
    dst = array
    if is_rgb:
        if np.random.random() < 0.5:
            dst = gaussian(array, sigma=abs(1. + 0.1 * np.random.randn()),preserve_range=True,multichannel=True)
    return dst
pre_process.append(gaussian_blur)

def multiple_transform(array, is_rgb):
    scalex = scaley = 1. + np.random.randn() * 0.1
    shift_y, shift_x = np.array(array.shape[:2]) / 2.
    shift = st(translation=[-shift_x, -shift_y])
    shift_inv = st(translation=[shift_x + np.random.randn() * 0., shift_y + np.random.randn() * 0.])
    trans = st(rotation=np.deg2rad(np.random.uniform(-90, 90)), scale=(scalex, scaley))
    final_transform = (shift + (trans + shift_inv)).inverse
    
    dst = warp(array, final_transform, mode='constant', order=0, cval=0, preserve_range=True)
    if is_rgb:
        dst = warp(array, final_transform, mode='reflect', order=1, cval=0, preserve_range=True)
    return dst
pre_process.append(multiple_transform)

In [11]:
""" normalization """
def normalization(array, is_rgb):
    dst = array
    if is_rgb:
      dst = (dst - 0.5) / 0.5
    return dst

pre_process.append(normalization)

# **Nerual Network (Unet)**

In [12]:
import torch.nn as nn
import torch

In [13]:
class unet(nn.Module):
    def __init__(self):
        super(unet,self).__init__()
        # input image
        self.inputLayer = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        # down layers
        self.downLayer1 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True)
        )
        self.downLayer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True)
        )
        self.downLayer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.MaxPool2d(2, 2),
            nn.ReLU(inplace=True)
        )

        # up layers
        self.upLayer3_1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upLayer3_2 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True)
        )

        self.upLayer2_1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.upLayer2_2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True)
        )

        self.upLayer1_1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.upLayer1_2 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ELU(inplace=True)
        )

        # output layer
        self.outputLayer = nn.Conv2d(32, 9, kernel_size=1, stride=1)

        # combine

    def forward(self,image):
        input = self.inputLayer(image)

        # down
        down1 = self.downLayer1(input)
        down2 = self.downLayer2(down1)
        down3 = self.downLayer3(down2)

        # up
        up3_0 = self.upLayer3_1(down3)
        up3_1 = torch.cat([up3_0, down2], dim=1)
        up3 = self.upLayer3_2(up3_1)

        up2_0 = self.upLayer2_1(up3)
        up2_1 = torch.cat([up2_0, down1], dim=1)
        up2 = self.upLayer2_2(up2_1)

        up1_0 = self.upLayer1_1(up2)
        up1_1 = torch.cat([up1_0, input], dim=1)
        up1 = self.upLayer1_2(up1_1)

        # output
        output = self.outputLayer(up1)

        return output

# **Loss function**

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
class HaloLoss(nn.Module):
  def __init__(self):
    super(HaloLoss,self).__init__()
  
  def forward(self, ypred, label):
    max_leaves = 30   # assume the maximum number of leaves
    k = np.array([[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0], # a halo kernel
            [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
            [0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0],
            [0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
            [0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
            [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
            [0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
            [0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
            [0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0],
            [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0]], dtype='float32')
    k_muti = np.expand_dims(np.expand_dims(k, 0), 0)
    kernel = np.vstack([k_muti]*max_leaves)
    kernel = torch.from_numpy(kernel).to(device)

    
    # split the leaves
    min_area = 10 # assume the area of leaf is less than it, then it is noise
    codeOfLeaves = []
    eachLeaf = np.zeros((label.shape[0], max_leaves, label.shape[1], label.shape[2]))

    for i in range(label.shape[0]):
      area = np.bincount(label[i].flatten())
      leaves = np.where(area > min_area)[0] # array: code of leaves and background
      max_index = 0
      for index, code in enumerate(leaves):
        # try:
          eachLeaf[i, index, label[i]==code] = float(1)
          max_index = index
        # except:
        #   print(i, len(eachLeaf), len(leaves))
        #   import sys
        #   sys.exit(1)
      leavesCode = [j for j in range(max_index + 1)]
      codeOfLeaves.append(np.array(leavesCode))
    eachLeaf =  torch.from_numpy(eachLeaf).float().to(device) # split the leaves
    
    # make a mask
    halo = F.conv2d(eachLeaf, kernel, groups=max_leaves, padding=int(k.shape[0]/2)) # create a halo mask
    halo[halo > 0] = float(1)     # halo mask = 1
    
    halo[eachLeaf > 0] = float(2)   # leaves = 2
    halo[:, 0, :, :] = float(1)   # backgound = 1
    
    halo_mask = halo.sum(3 ,keepdim=True)
    halo_mask = halo_mask.sum(2, keepdim=True)
    halo_mask[halo_mask == 0] = 1
    halo = halo / halo_mask

    # calculate loss
    y_log = F.log_softmax(ypred, dim=1).float()
    y_soft = F.softmax(ypred, dim=1).float()
    lossValue = torch.tensor([0]).float().to(device)  # initial a loss = 0
    
    for i in range(ypred.shape[0]):
      background = halo[i, 0] * eachLeaf[i, 0]
      lossValue -= torch.sum(background * y_log[i, 0]) / torch.sum(background)
            
      if len(codeOfLeaves[i]) == 1: # only backgound, no leaves
        continue
      index = torch.LongTensor(codeOfLeaves[i][1:]).to(device) # create an index of leaves
      valid_leaves = eachLeaf[i].index_select(0, index)
      valid_masks = halo[i].index_select(0, index)
    
      target_pos = valid_leaves * valid_masks
      target_neg = (1. - valid_leaves) * valid_masks

      positive = -torch.mm(y_log[i,1:].view(y_log[i,1:].shape[0],-1),target_pos.view(target_pos.shape[0],-1).transpose(1,0))/torch.sum(target_pos)
      negative = -torch.mm(torch.log(1-y_soft[i, 1:]+1e-12).view(torch.log(1-y_soft[i, 1:]+1e-12).shape[0], -1), 
                           target_neg.view(target_neg.shape[0],-1).transpose(1,0)) / torch.sum(target_neg)
      _, index_ = torch.min(positive + 7 * negative, 0)
      lossValue += torch.sum(positive[index_, torch.arange(0, positive.shape[1]).long().to(device)])   
    lossValue = lossValue * (1000/(ypred.shape[2] * ypred.shape[3]))
    return lossValue

# **Train**

In [16]:
from sys import stdout
import os
import numpy as np
import torch
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelName = 'drive/My Drive/COMP9517/ALL/ALL model C9_'

In [17]:
model = unet().cuda()
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lossFunc = HaloLoss()

In [18]:

epochs = 5000
loopSize = 1 
batchSize = 30
for epoch in range(epochs):
  # do preprocess and generate a images and labels list
    X, y = [], []
    for k in range(loopSize):
        r_index = list(range(len(images)))
        random.shuffle(r_index)
        # print(r_index)
        assert len(images) == len(labels)
        for i in r_index[:batchSize]:    # traverse all images
            ximg = images[i]
            ximg = transform.resize(ximg, np.array(ximg.shape[:2]) / 2,   mode='constant')
            ylabel = labels[i]
            ylabel = cv2.resize(ylabel, (ylabel.shape[0] // 2, ylabel.shape[1] // 2), interpolation=cv2.INTER_NEAREST)


            for approach in pre_process:  # do all preprocess to an image
                seed = np.random.randint(0, 2**32-1)
                np.random.seed(seed)
                ximg = approach(ximg, True)
                np.random.seed(seed)
                ylabel = approach(ylabel, False)
            

            ximg = ximg.transpose(2, 0, 1)
            ximg = np.expand_dims(ximg, 0)
            ylabel = np.expand_dims(ylabel, 0)
            y.append(ylabel)
            X.append(ximg)
    X = np.vstack(X).astype("float32")
    y = np.vstack(y).astype("int32")
  
  # train
    Xtrain = torch.from_numpy(X).to(device) # np to tensor
    ytrain = torch.from_numpy(y)

    optimizer.zero_grad()
    # print(Xtrain.size(), ytrain.size())
    ypred = model(Xtrain)      # get output
    loss = lossFunc(ypred, ytrain) # calculate loss value
    loss.backward()    # calculate gradients
    optimizer.step()   # Minimise the loss according to the gradient.

    if epoch % 5 == 0:
        p = epoch/epochs
        print("process: %.2f%% (%d in %d)"%(p*100, epoch+1, epochs))

    if (epoch+1) in {50, 100, 200, 500, 1000, 2000, 5000}:
        torch.save(model.state_dict(), modelName + str(epoch) + '_LZX.t7')
        print("%s Save successfully."%(epoch+1))


p = (epoch+1)/epochs
print("process: %.2f%% (%d in %d)"%(p*100, epoch+1, epochs))
torch.save(model.state_dict(), modelName + str(epochs) + '_LZX.t7')
print("Save successfully. Finish")


process: 0.00% (1 in 5000)
process: 0.10% (6 in 5000)
process: 0.20% (11 in 5000)
process: 0.30% (16 in 5000)
process: 0.40% (21 in 5000)
process: 0.50% (26 in 5000)
process: 0.60% (31 in 5000)
process: 0.70% (36 in 5000)
process: 0.80% (41 in 5000)
process: 0.90% (46 in 5000)
50 Save successfully.
process: 1.00% (51 in 5000)
process: 1.10% (56 in 5000)
process: 1.20% (61 in 5000)
process: 1.30% (66 in 5000)
process: 1.40% (71 in 5000)
process: 1.50% (76 in 5000)
process: 1.60% (81 in 5000)
process: 1.70% (86 in 5000)
process: 1.80% (91 in 5000)
process: 1.90% (96 in 5000)
100 Save successfully.
process: 2.00% (101 in 5000)
process: 2.10% (106 in 5000)
process: 2.20% (111 in 5000)
process: 2.30% (116 in 5000)
process: 2.40% (121 in 5000)
process: 2.50% (126 in 5000)
process: 2.60% (131 in 5000)
process: 2.70% (136 in 5000)
process: 2.80% (141 in 5000)
process: 2.90% (146 in 5000)
process: 3.00% (151 in 5000)
process: 3.10% (156 in 5000)
process: 3.20% (161 in 5000)
process: 3.30% (166 